In [57]:
from bs4 import BeautifulSoup as bs
import re 
import os
import lxml
import xml.etree.ElementTree as ET
from lxml import objectify
from datetime import datetime
import numpy as np 
import pandas as pd 
import requests
import ftplib
from io import BytesIO
import tqdm

### Prepare to load data

In [3]:
'''
with open(file) as f:
    parsed = objectify.parse(f)
root = parsed.getroot()
'''

'\nwith open(file) as f:\n    parsed = objectify.parse(f)\nroot = parsed.getroot()\n'

##### Read xml file 

In [6]:
file = "20091_M100_helemoedet.xml"
tree = ET.parse(file)
root = tree.getroot()

##### Subelements / children

In [8]:
# children of the root only 
root_children = []

for child in root:
    root_children.append((child.tag, child.attrib))

In [9]:
# All children in file
all_children = [elem.tag for elem in root.iter()]

In [10]:
unique_children = set(all_children)
print(unique_children)

{'MeetingNumber', 'Taler', 'GroupNameShort', 'TalerTitel', 'DateOfSitting', 'Char', 'Rubrica', 'ParliamentarySession', 'AudioFileFolder', 'PunktTekst', 'Dokument', 'ParliamentaryGroup', 'Titel', 'Linea', 'Location', 'MetaFTAgendaItem', 'FTCase', 'Exitus', 'ShortTitle', 'PreTekst', 'OratorFirstName', 'DagsordenPlan', 'Aktivitet', 'TitelGruppe', 'FTCaseType', 'OratorLastName', 'OratorRole', 'FTCaseStage', 'MetaMeeting', 'TekstGruppe', 'StartDateTime', 'TaleType', 'DagsordenPunkt', 'FTCaseNumber', 'MetaSpeakerMP', 'ItemNo', 'TaleSegment', 'EdixiStatus', 'Tale', 'LastModified', 'EdixiDocLocation', 'MetaSpeechSegment', 'EndDateTime', 'UnderTitel'}


# Kim

In [58]:
ftp_url = "oda.ft.dk"
ftp_dir = "/ODAXML/Referat/samling/"

# Initialisering af lister
### Jeg skal finde ud af, hvordan jeg dropper dem igen, når jeg er færdig

In [59]:
dirs = []
sub_dir = []
filer = []
filer_navne = []
sti_navn = []
indhold = []

suppe = []

### Login til ftp serveren og henter en liste over biblioteker

In [60]:
ftp = ftplib.FTP(ftp_url)
ftp.login("anonymous", "wpg345@alumni.ku.dk")
ftp.set_pasv(True)
ftp.cwd(ftp_dir)
ftp.dir(dirs.append)

### Bygger en liste med underbibliotekter

In [61]:
for i in range (0, len(dirs), 1):
    sub_dir.append(ftp_dir+dirs[i][-5:]+'/') 

### Bygger en liste med filnavne

In [62]:
for i in range (0, len(sub_dir), 1):
    ftp.cwd(sub_dir[i])
    ftp.dir(filer.append)

### Ekstrakt af filnavn

In [63]:
for i in range(0, len(filer)):
    filer_navne.append(filer[i][39:])

### Opbygning af hele stien og filnavn

In [64]:
for i in range(0, len(filer_navne)):
    sti_navn.append(ftp_dir+filer_navne[i][0:5]+'/'+filer_navne[i])

### Overførsel af de enkelte filer fra ftp://oda.ft.dk, og indholdet skrives til liste

In [65]:
for i in tqdm.tqdm(range(0, len(sti_navn))):
# for i in tqdm.tqdm(range(0, 10)):
    r = BytesIO()
    ftp.retrbinary(f"RETR {sti_navn[i]}", r.write)
    indhold.append(r.getvalue())
    r.close()

100%|██████████| 1489/1489 [02:53<00:00,  8.58it/s]


# Single file (lokal)

In [12]:
# entire xml file as string
xml = ET.tostring(root, encoding='utf8').decode('utf8')

##### Soup 

In [14]:
soup = bs(xml, 'lxml')

In [15]:
dagsordenpunkter = soup.find_all("dagsordenpunkt") # punkterne med tags 
len(dagsordenpunkter)

8

In [ ]:
# dagsordenpunkter[1]

### Remove datetime tags (lokal)

In [18]:
'''
for tag in soup.find_all('startdatetime'):
    tag.replace('')
    
for tag in soup.find_all('enddatetime'):
    tag.replace('')

for tag in soup.find_all('lastmodified'):
    tag.replace('')
    
for tag in soup.find_all('edixistatus'):
    tag.replace('')
'''

"\nfor tag in soup.find_all('startdatetime'):\n    tag.replace('')\n    \nfor tag in soup.find_all('enddatetime'):\n    tag.replace('')\n\nfor tag in soup.find_all('lastmodified'):\n    tag.replace('')\n    \nfor tag in soup.find_all('edixistatus'):\n    tag.replace('')\n"

In [19]:
'''
soup.find_all('startdatetime').decompose()
soup.find_all('enddatetime').decompose()
soup.find_all('lastmodified').decompose()
'''

"\nsoup.find_all('startdatetime').decompose()\nsoup.find_all('enddatetime').decompose()\nsoup.find_all('lastmodified').decompose()\n"

In [20]:
soup.startdatetime.clear()
soup.enddatetime.clear()
soup.lastmodified.clear()

### Dagsordenpunkter uden tags (lokal)

In [22]:
punkter = []  # punkterne uden tags 

for dagsordenpunkt in range(0, len(dagsordenpunkter)):
    punkter.append(dagsordenpunkter[dagsordenpunkt].get_text(separator = ' '))

In [23]:
punkter[0]

'0 FM Punkt 0 Meddelelser fra formanden Holger K. Nielsen SF formand Tredje næstformand  (Holger K. Nielsen) Typeset Mødet er åbnet. Finansudvalget har afgivet: Betænkning og indstilling vedrørende forslag til folketingsbeslutning i henhold til grundlovens § 47 med hensyn til statsregnskabet for finansåret 2008. (Beslutningsforslag nr. B 248). Betænkningen og indstillingen vil fremgå af www.folketingstidende.dk.'

### Meta data (lokal)

In [25]:
dato = bs(xml).find("dateofsitting").text[:10]
moedenr = bs(xml).find("meetingnumber").text
samling = bs(xml).find("parliamentarysession").text
hvem = bs(xml).find("parliamentarygroup").text
lokation = bs(xml).find("location").text

In [26]:
lokation

'Folketingssalen'

### List of filepaths (skrottet)

In [28]:
def getListOfFiles(dirName):
    listOfFile = os.listdir(dirName)
    allFiles = list()
    for entry in listOfFile:
        fullPath = os.path.join(dirName, entry)
        if os.path.isdir(fullPath):
            allFiles = allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)
                
    return allFiles 

#getListOfFiles('all')

In [29]:
path_list = getListOfFiles('sample')

In [66]:
path_list

['sample/20171_M11_helemoedet.xml',
 'sample/20211_M60_helemoedet.xml',
 'sample/20171_M46_helemoedet.xml']

# Extract metadata function

In [69]:
#def extract_metadata(file): 
    #tree = ET.parse(file)
    #root = tree.getroot()
    #xml = ET.tostring(root, encoding='utf8').decode('utf8')

def extract_metadata(xml): 
    soup = bs(xml, 'lxml')
    
    d = soup.find("dateofsitting").text[:10]
    m = soup.find("meetingnumber").text
    s = soup.find("parliamentarysession").text
    g = soup.find("parliamentarygroup").text
    l = soup.find("location").text
    
    date.append(d)
    meetingnumber.append(m)
    session.append(s)
    group.append(g)
    location.append(l)

In [70]:
# empty lists (columns)
date = []
content = []
meetingnumber = []
session = []
group = []
location = []

# Content function 

In [88]:
''' lokalt
def extract_content(file):
    tree = ET.parse(file)
    root = tree.getroot()
    xml = ET.tostring(root, encoding='utf8').decode('utf8')
'''    

def extract_content(xml):
    soup = bs(xml, 'lxml')
    
    dp = soup.find_all("dagsordenpunkt")
    
    for punkt in range(0, len(dp)):
        content.append(dp[punkt].get_text(separator = ' '))

# Apply content function to files

In [37]:
extract_content('20091_M100_helemoedet.xml')

In [38]:
dp = soup.find_all("dagsordenpunkt")
    
for punkt in range(0, len(dp)):
    content.append(dp[punkt].get_text(separator = ' '))

In [87]:
len(content)

0

In [40]:
t = extract_content('20091_M100_helemoedet.xml')

In [41]:
for path in path_list: 
    extract_content(path)

# Apply metadata function to files

In [43]:
#for path in path_list: 
#    extract_metadata(path)

In [85]:
test = indhold[0:5]
for i in test: 
    extract_metadata(i)

In [86]:
session

['20091',
 '20091',
 '20091',
 '20091',
 '20091',
 '20091',
 '20091',
 '20091',
 '20091',
 '20091',
 '20091',
 '20091']

# Make DataFrame

In [46]:
df = pd.DataFrame([date, content, meetingnumber, session, group, location]).transpose()
df.columns = ['date', 'content', 'meetingnumber', 'session', 'group', 'location']
df

,date,content,meetingnumber,session,group,location
0,2017-10-27,0 FM Punkt 0 Meddelelser fra formanden Holger ...,11,20171,Folketinget,Folketingssalen
1,2022-02-10,1 222 L BEH1 1. behandling af L 222: Om nedsæt...,60,20211,Folketinget,Folketingssalen
2,2018-01-18,2 218 L BEH1 1. behandling af L 218: Om nedsæt...,46,20171,Folketinget,Folketingssalen
3,None,3 219 L BEH1 1. behandling af L 219: Om forhøj...,None,None,None,None
4,None,4 223 L BEH1 1. behandling af L 223: Om staten...,None,None,None,None
5,None,5 221 L BEH1 1. behandling af L 221: Om person...,None,None,None,None
6,None,6 220 L BEH1 1. behandling af L 220: Om nedsæt...,None,None,None,None
7,None,0 FM Punkt 0 Meddelelser fra formanden Holger ...,None,None,None,None
